## Imports, Configs, and Constants

In [1]:
import polars as pl
import json
from web3 import Web3
from eth_abi import decode as decode_abi
from eth_account import Account
import math

mainnet = Web3(Web3.HTTPProvider("https://eth.llamarpc.com"))
anvil = Web3(Web3.HTTPProvider("http://localhost:8545"))

Account.enable_unaudited_hdwallet_features()
account = anvil.eth.account.from_mnemonic(
    "test test test test test test test test test test test junk"
)

default_txn = {
    'from': account.address,
    'gas': 2_000_000,
    'maxFeePerGas': anvil.to_wei(10, 'gwei'),
    'maxPriorityFeePerGas': anvil.to_wei(2, 'gwei'),
}

# parameters for infinite impact
MIN_PRICE_LIMIT = 4295128739 + 1
MAX_PRICE_LIMIT = 1461446703485210103287273052203988822378723970342 - 1

In [2]:
!forge script ../Anvil.s.sol --rpc-url http://localhost:8545 --private-key 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80 --code-size-limit 30000 --broadcast --silent


###
Finding wallets for all the necessary addresses...
⠄ [00:00:00] [###########################################] 16/16 receipts (0.0s)⠁ [00:00:00] [--------------------------------------------] 0/16 receipts (0.0s)
##### anvil-hardhat
✅  [Success]Hash: 0xed778187494fb29329a488719545f9e42949f46a31b1cc7401367ba2abebbd56
Contract Address: 0x5FbDB2315678afecb367f032d93F642f64180aa3
Block: 1
Paid: 0.02424584 ETH (6061460 gas * 4 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0xeb90c9e33e7dcc6d7371b8bbdf81b944cb4f785b910804c5f71165f6ffed56a7
Contract Address: 0xe7f1725E7734CE288F8367e1Bb143E90bb3F0512
Block: 1
Paid: 0.006136544 ETH (1534136 gas * 4 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0xd2d77a3d38f6e8f57175105d6a24cb5055d789e49c64dd6c8816ef313bfb59ec
Contract Address: 0x9fE46736679d2D9a65F0992F2272dE9f3c7fa6e0
Block: 1
Paid: 0.001877104 ETH (469276 gas * 4 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0xdb80197eda5172276e08fc3f0558f745afc9457ede0a3b7e7e644d0b33c94af7
Contract Addre

Uni v3 ETH/USDC 5bps Logs

In [3]:
df = pl.read_parquet("../../cryo_data/*.parquet")
print(df.shape)
print(df.head())
print(df.columns)

(56503, 10)
shape: (5, 10)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬────────┬───────────┐
│ block_numb ┆ transactio ┆ log_index ┆ transacti ┆ … ┆ topic1    ┆ topic2    ┆ topic3 ┆ data      │
│ er         ┆ n_index    ┆ ---       ┆ on_hash   ┆   ┆ ---       ┆ ---       ┆ ---    ┆ ---       │
│ ---        ┆ ---        ┆ u32       ┆ ---       ┆   ┆ binary    ┆ binary    ┆ binary ┆ binary    │
│ u32        ┆ u32        ┆           ┆ binary    ┆   ┆           ┆           ┆        ┆           │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪════════╪═══════════╡
│ 17686542   ┆ 0          ┆ 2         ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]     ┆        ┆ data]     │
│ 17686543   ┆ 84         ┆ 194       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]   

Get topic hashes for desired logs: Mint, Burn, and Swap

In [4]:
!cast sig-event "Mint(address sender, address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Burn(address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Swap(address indexed sender, address indexed recipient, int256 amount0, int256 amount1, uint160 sqrtPriceX96, uint128 liquidity, int24 tick)"

0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde
0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67


In [5]:
mint_event_bytes = Web3.to_bytes(0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde)
burn_event_bytes = Web3.to_bytes(0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c)
swap_event_bytes = Web3.to_bytes(0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67)

event_df = df.filter((pl.col("topic0") == mint_event_bytes) | (pl.col("topic0") == burn_event_bytes) | (pl.col("topic0") == swap_event_bytes))
print("All events: ", event_df.shape)
print("Mint events: ", event_df.filter(pl.col("topic0") == mint_event_bytes).shape)
print("Burn events: ", event_df.filter(pl.col("topic0") == burn_event_bytes).shape)
print("Swap events: ", event_df.filter(pl.col("topic0") == swap_event_bytes).shape)
event_df.head(8)

All events:  (55483, 10)
Mint events:  (968, 10)
Burn events:  (1056, 10)
Swap events:  (53459, 10)


block_number,transaction_index,log_index,transaction_hash,contract_address,topic0,topic1,topic2,topic3,data
u32,u32,u32,binary,binary,binary,binary,binary,binary,binary
17686542,0,2,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686543,84,194,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686543,100,218,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686545,13,33,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686546,161,215,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686548,0,2,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686549,11,28,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17686549,130,250,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]


In [6]:
mint_decode_types = ["address", "uint128", "uint256", "uint256"]
burn_decode_types = ["uint128", "uint256", "uint256"]
swap_decode_types = ["int256", "int256", "uint160", "uint128", "int24"]

## Instantiate Contracts

In [7]:
with open("../../broadcast/Anvil.s.sol/31337/run-latest.json", "r") as f:
    deployment = json.load(f)

contract_creates = list(filter(lambda txn: txn['transactionType'] == 'CREATE', deployment['transactions']))

contract_names = [
    'MockERC20',
    'PoolManager',
    'PoolModifyPositionTest',
    'PoolSwapTest',
    'PoolDonateTest',
    'AtomicArb'
]

contracts = {}
for contract_name in contract_names:
    with open(f"../../out/{contract_name}.sol/{contract_name}.json", "r") as f:
        abi = json.load(f)['abi']

    address = list(filter(lambda txn: txn['contractName'] == contract_name, contract_creates))[0]['contractAddress']
    print(f"{address}\t{contract_name}")
    contracts[contract_name] = anvil.eth.contract(address=address, abi=abi)

with open(f"../../out/MockERC20.sol/MockERC20.json", "r") as f:
    abi = json.load(f)['abi']

# WETH
contracts['MWETH'] = anvil.eth.contract(address="0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", abi=abi)
print(f"{contracts['MWETH'].address}\tMock WETH")

# USDC
contracts['MUSDC'] = anvil.eth.contract(address="0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", abi=abi)
print(f"{contracts['MUSDC'].address}\tMock USDC")


0x0165878A594ca255338adfa4d48449f69242Eb8F	MockERC20


0x5FbDB2315678afecb367f032d93F642f64180aa3	PoolManager
0xCf7Ed3AccA5a467e9e704C703E8D87F634fB0Fc9	PoolModifyPositionTest
0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9	PoolSwapTest
0x5FC8d32690cc91D4c39d9d3abcBD16989F875707	PoolDonateTest
0xe7f1725E7734CE288F8367e1Bb143E90bb3F0512	AtomicArb
0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2	Mock WETH
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48	Mock USDC


Define Pool Keys


In [8]:
currency0 = contracts['MUSDC'].address if anvil.to_int(hexstr=contracts['MUSDC'].address) < anvil.to_int(hexstr=contracts['MWETH'].address) else contracts['MWETH'].address
currency1 = contracts['MUSDC'].address if currency0 == contracts['MWETH'].address else contracts['MWETH'].address
print(currency0, currency1)

key0 = {
    "currency0": currency0,
    "currency1": currency1,
    "fee": 500,
    "tickSpacing": 10,
    "hooks": "0x0000000000000000000000000000000000000000"
}

key1 = {
    "currency0": currency0,
    "currency1": currency1,
    "fee": 500,
    "tickSpacing": 10,
    "hooks": "0x0c00000000000000000000000000000000000001"
}

0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


Initialize Pools

* Take the first `Mint()` event to get `sqrtPriceRatioX96`

In [9]:
mint_df = event_df.filter(pl.col("topic0") == mint_event_bytes)
(_, amount, amount0, amount1) = decode_abi(mint_decode_types, mint_df[0, 'data'])

tickLower = decode_abi(["int24"], mint_df[0, 'topic2'])
tickUpper = decode_abi(["int24"], mint_df[0, 'topic3'])

sqrtPriceX96 = math.floor(math.sqrt(amount1/amount0)*2**96)

# initialize pools
contracts['PoolManager'].functions['initialize'](
    key0, sqrtPriceX96
).transact(default_txn)

contracts['PoolManager'].functions['initialize'](
    key1, sqrtPriceX96
).transact(default_txn)


HexBytes('0x4f6430b68ebc40ef6d63b5f5785d2ba586b1f14e05471149fc50961b6fb2db78')

## Simulation

In [16]:
def modifyPosition(poolKey, tickLower, tickUpper, liquidityDelta):
    positionParams = {
        "tickLower": tickLower,
        "tickUpper": tickUpper,
        "liquidityDelta": liquidityDelta,
    }
    txn_hash = contracts['PoolModifyPositionTest'].functions["modifyPosition"](
        poolKey,
        positionParams
    ).transact(default_txn)
    return anvil.eth.wait_for_transaction_receipt(txn_hash)
    
def swap(poolKey, zeroForOne, amountSpecified):
    settings = {
        "withdrawTokens": True,
        "settleUsingTransfer": True
    }
    params = {
        "zeroForOne": zeroForOne,
        "amountSpecified": amountSpecified,
        "sqrtPriceLimitX96": MIN_PRICE_LIMIT if zeroForOne else MAX_PRICE_LIMIT,
    }
    txn_hash = contracts['PoolSwapTest'].functions['swap'](
        poolKey,
        params,
        settings
    ).transact(default_txn)
    return anvil.eth.wait_for_transaction_receipt(txn_hash)

def get_arb_tx(key0, zeroForOne, amount, key1, takeToken0):
    return contracts['AtomicArb'].functions['arb'](
        key0,
        {
            "zeroForOne": zeroForOne,
            "amountSpecified": amount,
            "sqrtPriceLimitX96": MIN_PRICE_LIMIT if zeroForOne else MAX_PRICE_LIMIT,
        },
        key1,
        takeToken0
    )

Provision Liquidity to Primary Pool

In [11]:
# full range provision, to seed liquidity for initial swaps
before = contracts['MWETH'].functions['balanceOf'](account.address).call()
before_usdc = contracts['MUSDC'].functions['balanceOf'](account.address).call()
modifyPosition(key0, -887270, 887270, 10**17)
after = contracts['MWETH'].functions['balanceOf'](account.address).call()
after_usdc = contracts['MUSDC'].functions['balanceOf'](account.address).call()
print("WETH balance change: ", (after - before)/1e18)
print("USDC balance change: ", (after_usdc - before_usdc)/1e6)

WETH balance change:  -2284.92577313695
USDC balance change:  -4376509.783192


In [12]:
# full range provision, to seed liquidity for initial swaps
before = contracts['MWETH'].functions['balanceOf'](account.address).call()
before_usdc = contracts['MUSDC'].functions['balanceOf'](account.address).call()
modifyPosition(key1, -887270, 887270, 10**15)
after = contracts['MWETH'].functions['balanceOf'](account.address).call()
after_usdc = contracts['MUSDC'].functions['balanceOf'](account.address).call()
print("WETH balance change: ", (after - before)/1e18)
print("USDC balance change: ", (after_usdc - before_usdc)/1e6)

WETH balance change:  -22.849257731369498
USDC balance change:  -43765.097832


In [13]:
# loop over the events and simulate the actions against the pools
success = 0
fails = 0
for row in event_df[:50].iter_rows(named=True):
    if row['topic0'] == mint_event_bytes:
        (sender, amount, amount0, amount1) = decode_abi(mint_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        (tickLower,) = decode_abi(["int24"], row['topic2'])
        (tickUpper,) = decode_abi(["int24"], row['topic3'])
        txn = modifyPosition(key0, tickLower, tickUpper, amount)
        if txn['status'] == 1:
            success += 1
        else:
            fails += 1
    elif row['topic0'] == burn_event_bytes:
        (amount, amount0, amount1) = decode_abi(burn_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        (tickLower,) = decode_abi(["int24"], row['topic2'])
        (tickUpper,) = decode_abi(["int24"], row['topic3'])
        # TODO: figure out how to remove liquidity, since the burns arent 1:1 with the mints
        # modifyPosition(key0, tickLower, tickUpper, -amount)
    elif row['topic0'] == swap_event_bytes:
        (amount0, amount1, sqrtPriceX96, liquidity, tick) = decode_abi(swap_decode_types, row['data'])
        sender = decode_abi(["address"], row['topic1'])
        recipient = decode_abi(["address"], row['topic2'])
        # when amount1 < 0, the user traded token0 for token1
        zeroForOne = True if amount1 < 0 else False
        amountSpecified = amount0 if zeroForOne else amount1
        txn = swap(key0, zeroForOne=zeroForOne, amountSpecified=amountSpecified)
        if txn['status'] == 1:
            success += 1
        else:
            fails += 1

In [31]:
swap(key0, zeroForOne=False, amountSpecified=10**20)

AttributeDict({'transactionHash': HexBytes('0x2ef38212312f32ccfff0e33c6b8dd46f70bf4436ebc6db582e95c99f6ddbd4c0'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x23baad9423837766cf039b94b14ffed74dbfbfc975ead10fac48ec0c4a19ae79'),
 'blockNumber': 59,
 'from': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'to': '0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9',
 'cumulativeGasUsed': 190456,
 'gasUsed': 190456,
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x5FbDB2315678afecb367f032d93F642f64180aa3',
   'topics': [HexBytes('0x40e9cecb9f5f1f1c5b9c97dec2917b7ee92e57ba5563708daca94dd84ad7112f'),
    HexBytes('0x4f88f7c99022eace4740c6898f59ce6a2e798a1e64ce54589720b7153eb224a7'),
    HexBytes('0x000000000000000000000000dc64a140aa3e981100a9beca4e685f962f0cf6c9')],
   'data': HexBytes('0xffffffffffffffffffffffffffffffffffffffffffffffffffffffc236401c920000000000000000000000000000000000000000000000056bc75e2d631000000000000000000000000000000000000000004dc936f2a07489678251f9d108010000

In [33]:
arb_tx = get_arb_tx(key1, True, 100, key0, True)
arb_tx.transact(default_txn)

HexBytes('0xe85fa7b241b80b5f4d9b9f9eece9e9a83224b8ff1c813d11e6d5b46fa7f56de3')